In [1]:
from product import ProductRfrFuture, ProductOvernightCapFloor, RfrFutureVisitor, OvernightCapFloorVisitor
from yield_curve import YieldCurve
from sabr import SabrModel
from valuation import ValuationEngineRegistry, IndexManager
from analytics import SABRCalculator
from data import Data1D, Data2D, DataCollection
from date import Date
import pandas as pd

In [2]:
curve_data = [
    ["SOFR-1B",            "1M",  0.0020],
    ["SOFR-1B",            "3M",  0.0025],
    ["SOFR-1B",            "6M",  0.0030],
]
df_curve = pd.DataFrame(curve_data, columns=["INDEX","AXIS1","VALUES"])

sabr_data = pd.DataFrame([
    ["SOFR-1B", 0.25, 0.25, 0.010, 0.5, 0.2, -0.1],
    ["SOFR-1B", 0.25, 1.00, 0.012, 0.5, 0.2, -0.1],
    ["SOFR-1B", 1.00, 0.25, 0.013, 0.5, 0.2, -0.1],
    ["SOFR-1B", 1.00, 1.00, 0.015, 0.5, 0.2, -0.1],
], columns=["INDEX","AXIS1","AXIS2","NORMALVOL","BETA","NU","RHO"])

In [3]:
data_objs = []
d1 = Data1D.createDataObject(
    data_type       = "zero_rate",
    data_convention = "SOFR-1B",
    df              = df_curve[["AXIS1","VALUES"]]
)
data_objs.append(d1)

dc = DataCollection(data_objs)

for idx_name, sub in sabr_data.groupby("INDEX"):
    for param in ["NORMALVOL","BETA","NU","RHO"]:
        pivot = (
            sub
            .pivot(index="AXIS1", columns="AXIS2", values=param)
            .sort_index(axis=0).sort_index(axis=1)
        )
        d2 = Data2D.createDataObject(
            data_type       = param.lower(),
            data_convention = idx_name,
            df              = pivot
        )
        data_objs.append(d2)

dc = DataCollection(data_objs)

In [4]:
value_date = "2025-09-12"

build_method_yc = [
    {
        "TARGET" : "SOFR-1B",
        "INTERPOLATION METHOD" : "PIECEWISE_CONSTANT"
    }
]

yc = YieldCurve(value_date, dc, build_method_yc)


In [5]:
rfr_future = ProductRfrFuture(effectiveDate= "2026-09-12", termOrEnd="3M", index= "SOFR-1B", compounding="AVERAGE", strike=98.75, notional= 10000, longOrShort="SHORT")

In [6]:
rfr_future.accept(RfrFutureVisitor())

,Attribute,Value
0,MaturityDate,2026-12-14
1,EffectiveDate,2026-09-12
2,TerminationDate,2026-12-14
3,AccrualFactor,0.255556
4,Compounding,AVERAGE
5,Index,SOFR-1B
6,Strike,98.75
7,Notional,10000
8,Currency,USD
9,LongOrShort,SHORT


In [7]:
ve = ValuationEngineRegistry().new_valuation_engine(yc, {}, rfr_future,)

ve.calculateValue()
ve.value

['USD', np.float64(-8943.510052475289)]

In [8]:
from product import ProductOvernightSwap

ois_swap = ProductOvernightSwap(effectiveDate="2026-09-12", maturityDate= "2028-09-12", frequency="3M", overnightIndex="SOFR-1B", spread= 0, fixedRate=0.03, notional=1000000, position="SHORT")

In [9]:
from product import OvernightSwapVisitor

ois_swap.accept(OvernightSwapVisitor())

,Attribute,Value
0,EffectiveDate,2026-09-12
1,MaturityDate,2028-09-12
2,FixedRate,0.03
3,Index,SOFR-1B
4,PayFixed,True
5,Notional,1000000
6,Currency,USD
7,LongOrShort,SHORT


In [10]:
ve1 = ValuationEngineRegistry().new_valuation_engine(yc,{"FUNDING INDEX": "SOFR-1B"},ois_swap)
ve1.calculateValue()
ve1.value

['USD', np.float64(-7999665.53299931)]

In [ ]:
ve1.parRateOrSpread()


np.float64(185.65917040950015)